In [1]:
import requests
import datetime
import time
import pulsar
from pulsar import PartitionsRoutingMode
from pulsar import MessageId
from pulsar import Function
import _pulsar

In [41]:
class PulsarConnection():

    def __init__(self):
        self.client = pulsar.Client('pulsar://localhost:6650')
        self.tenant = 'public'
        self.namespace = 'default'
        self.static_namespace = 'static'
        self.initializing = False
        self.initialized = False
        self.last_day_processed = False
        self._set_init_status() # updates 'initialized' and 'initializing'
        # Initialize the system if it hasn't
        if not self.initialized:
            self._initialize_pulsar()
    
    def close(self):
        """ Remeber to close when finished working """
        try: self.client.close()
        except Exception as e: print(f"\n*** Exception: {e} ***\n")
        
    def callback(self, res, msg_id):
        """ Needed for 'send_async', for when the broker receives the message """
        return

    def _set_init_status(self):
        """ Identifying the status based on messages of 'initialized' topic.
        If there are no messages its because it hasn't been initialized.
        A first 'Initializing' message is because its starting, and a second
        'Initialized' message is because it has already started
        """
        try:
            has_messages = False
            curr_time = str(int(time.time()))
            topic_name = 'initialized'
            reader = self.client.create_reader(
                topic=f'persistent://{self.tenant}/{self.static_namespace}/{topic_name}',
                reader_name=topic_name+'_read_'+curr_time,
                start_message_id=MessageId.earliest)
            has_messages = reader.has_message_available()
        except Exception as e: print(f"\n*** Exception: {e} ***\n")
        # If there are no messages, it hasn't been initialized
        if not has_messages:
            print("\nIt seems the system needs initializing\n")
            reader.close()
            return
    
        # Check now if it is initializing
        msg = reader.read_next()
        message = str(msg.value().decode())
        if message == "Initializing":
            print("Found 'Initializing' message")
            self.initializing = True
        else:
            print("Didn't found 'Initializing' message")
        
        # Check if it has already initialized
        try:
            msg = reader.read_next(timeout_millis=3000)
            message = str(msg.value().decode())
        except Exception as e:
            print(f"\n*** Didn't found 'Initilized' message : {e} ***\n")
            
        if message == "Initialized":
            print("Found 'Initialized' message")
            self.initialized = True
            self.initializing = False
        reader.close()
        
        return True
    
    def create_day_to_process(self):
        """ Create the 'day_to_process' topic with 365 'YYYY-MM-DD' values.
        To be called automatically while initializing, but also making it externally
        available for test purposes """
        print("\n*** Populating 'day_to_process' topic ***\n")
        try:
            curr_time = str(int(time.time()))
            topic_name = 'day_to_process'
            day_producer = self.client.create_producer(
                topic=f'persistent://{self.tenant}/{self.namespace}/{topic_name}',
                message_routing_mode=PartitionsRoutingMode.UseSinglePartition)
        except Exception as e:
            print(f"\n*** Exception creating 'day_to_process' topic: {e} ***\n")
            day_producer.close()
            return
        init_date = datetime.datetime(2021, 1, 1)
        dates = [(init_date + datetime.timedelta(days=idx)).strftime('%Y-%m-%d') for idx in range(365)]
        
        for date in dates:
            try:
                day_producer.send((f"{date}").encode('utf-8'))      
            except Exception as e:
                print(f"\n*** Exception sending date message: {e} ***\n")
                day_producer.close()
                return
        day_producer.close()
        return True
    
    def _initialize_pulsar(self):
        """ Initialize relevant topics: 'initialized', 'days_processed' """
        
        # If its initilizing elsewhere, wait until it has finished
        while self.initializing:
            print("\nThe system is initializing elsewhere, checking again in 5 seconds..\n")
            time.sleep(5)
            self._set_init_status()
            
        
        # Send an Initializing message, so other workers don't do the same
        print("\n*** Initializing the system *** \n")
        try:
            curr_time = str(int(time.time()))
            topic_name = 'initialized'
            init_producer = self.client.create_producer(
                topic=f'persistent://{self.tenant}/{self.static_namespace}/{topic_name}',
                producer_name=f'{topic_name}_prod_{curr_time}',
                message_routing_mode=PartitionsRoutingMode.UseSinglePartition)
        except Exception as e:
            print(f"\n*** Exception: {e} ***\n")
            init_producer.close()
            return
        
        try:
            init_producer.send(("Initializing").encode('utf-8'))      
        except Exception as e:
            print(f"\n*** Exception sending Initializing message: {e} ***\n")
            init_producer.close()
            return
        
        self.initializing = True
            
        self.create_day_to_process()
        
        try:
            init_producer.send(("Initialized").encode('utf-8'))      
        except Exception as e:
            print(f"\n*** Exception sending Initialized message: {e} ***\n")
            init_producer.close()
            return
        init_producer.close()
        
        self.initializing = False
        self.initialized = True
        print("\n*** The system has been initialized. Now get working! ***\n")
        
        return True
    
    def get_day_to_process(self):
        """ Pops a ‘YYYY-MM-DD’ string value from the topic 'day_to_process'.
        If there are no more days to process, returns None (Null) """
        if self.last_day_processed: return None
    
        topic_name = 'day_to_process'
        # Create a consumer on persistent topic, always using the same name,
        # so it always references to the current read position
        try:
            curr_time = str(int(time.time()))
            day_consumer = self.client.subscribe(
                topic=f"persistent://{self.tenant}/{self.namespace}/{topic_name}",
                subscription_name=f'{topic_name}_sub',
                initial_position=_pulsar.InitialPosition.Earliest)
        except Exception as e:
            print(f"\n*** Exception creating day_consumer: {e} ***\n")
            day_consumer.close()
            return
            
        try:
            msg = day_consumer.receive()
            # Save the string message (decode from byte value)
            message = str(msg.value().decode())
            # Acknowledge that the message was received          
            day_consumer.acknowledge(msg)
        except Exception as e:
            print(f"\n*** Exception receiving value from 'day_consumer': {e} ***\n")
            day_consumer.negative_acknowledge(msg)
            day_consumer.close()
            return
        
        # If we reached the end, signal so we start sending None from next call on
        if message == '2021-12-31':
            self.last_day_processed = True
        
        # Close at the end so no to block other processes that use the same subscription
        day_consumer.close()
        return message
            
    def get_initializing(self):
        return self.initializing
    
    def get_initialized(self):
        return self.initialized
    
    def put_basic_repo_info(self):
        """ Todo """
        return

    def get_basic_repo_info(self):
        """ Todo """
        return

In [43]:
my_pulsar = PulsarConnection()

2022-05-26 07:33:24.460 INFO  [140597701977920] ClientConnection:189 | [<none> -> pulsar://localhost:6650] Create ClientConnection, timeout=10000
2022-05-26 07:33:24.460 INFO  [140597701977920] ConnectionPool:96 | Created connection for pulsar://localhost:6650
2022-05-26 07:33:24.467 INFO  [140596878497536] ClientConnection:375 | [127.0.0.1:40234 -> 127.0.0.1:6650] Connected to broker
2022-05-26 07:33:24.468 INFO  [140596878497536] HandlerBase:64 | [persistent://public/static/initialized, reader-b64dad1236, 0] Getting connection from pool
2022-05-26 07:33:24.471 INFO  [140596878497536] ConsumerImpl:224 | [persistent://public/static/initialized, reader-b64dad1236, 0] Created consumer on broker [127.0.0.1:40234 -> 127.0.0.1:6650] 
Found 'Initializing' message
Found 'Initialized' message
2022-05-26 07:33:24.475 INFO  [140597701977920] ConsumerImpl:999 | [persistent://public/static/initialized, reader-b64dad1236, 0] Closing consumer for topic persistent://public/static/initialized
2022-05-

In [4]:
my_pulsar.get_initializing()

False

In [5]:
my_pulsar.get_initialized()

True

In [44]:
day = my_pulsar.get_day_to_process()

2022-05-26 07:33:28.498 INFO  [140597701977920] Client:88 | Subscribing on Topic :persistent://public/default/day_to_process
2022-05-26 07:33:28.500 INFO  [140597701977920] HandlerBase:64 | [persistent://public/default/day_to_process, day_to_process_sub, 1] Getting connection from pool
2022-05-26 07:33:28.506 INFO  [140597701977920] ConsumerImpl:224 | [persistent://public/default/day_to_process, day_to_process_sub, 1] Created consumer on broker [127.0.0.1:40234 -> 127.0.0.1:6650] 
2022-05-26 07:33:28.524 INFO  [140597701977920] ConsumerImpl:999 | [persistent://public/default/day_to_process, day_to_process_sub, 1] Closing consumer for topic persistent://public/default/day_to_process
2022-05-26 07:33:28.525 INFO  [140597701977920] ConsumerImpl:1055 | [persistent://public/default/day_to_process, day_to_process_sub, 1] Closed consumer 1


In [45]:
day

'2021-01-15'

In [46]:
my_pulsar.close()

2022-05-26 07:33:34.221 INFO  [140597701977920] ClientImpl:496 | Closing Pulsar client with 0 producers and 2 consumers
2022-05-26 07:33:34.222 INFO  [140597701977920] ClientConnection:1559 | [127.0.0.1:40234 -> 127.0.0.1:6650] Connection closed
2022-05-26 07:33:34.222 INFO  [140597701977920] ClientConnection:263 | [127.0.0.1:40234 -> 127.0.0.1:6650] Destroyed connection


# Test code below

In [ ]:
client = pulsar.Client('pulsar://localhost:6650')

try:
    curr_time = str(int(time.time()))
    topic_name = 'days_processed'
    reader = client.create_reader(
        topic=f'persistent://public/default/{topic_name}',
        start_message_id=MessageId.earliest,
        reader_name=topic_name+'_read_'+curr_time)
    print("\n\nAny messages around?: ", reader.has_message_available())
except Exception as e: print(f"\n*** Exception: {e} ***\n")

client.close()

In [ ]:
# Testing to see if we can check for a topic's existance.
# However, it might be better to do this using putstate
if False:
    test_client = pulsar.Client('pulsar://localhost:6650')
    persistent_path = 'persistent://public/default'

    try:
        curr_time = str(int(time.time()))
        topic_name = 'days_processed'
        consumer = test_client.subscribe(
            topic=f'{persistent_path}/{topic_name}',
            subscription_name=topic_name+'_sub_'+curr_time,
            initial_position=_pulsar.InitialPosition.Earliest)
    except Exception as e:
        print('Subscription exception:')
        print(e)

    msg = consumer.receive()

    try:
        # Save the string message (decode from byte value)
        message = str(msg.value().decode())
        # Acknowledge that the message was received
        consumer.acknowledge(msg)

        print(message)

    except Exception as e:
        print("Message reception exception")
        consumer.negative_acknowledge(msg)


    test_client.close()